In [147]:
from bs4 import BeautifulSoup
import urllib2
import selenium

In [247]:
#soup = BeautifulSoup(open("/Network/Servers/pg1.phys.columbia.edu/Users/Shared/vg2337/Downloads/ibm-20140630.xml"),"lxml-xml")
url = "https://www.sec.gov/cgi-bin/viewer?action=view&cik=51143&accession_number=0001047469-15-001106"
page = urllib2.urlopen(url)
soup = BeautifulSoup(page,"lxml")

In [251]:
for i in soup.find_all(class_='xbrlviewer'):
    print i.text,i.attrs['href']

Print Document javascript:window.print();
View Excel Document /Archives/edgar/data/51143/000104746915001106/Financial_Report.xlsx
Document and Entity Information javascript:loadReport(1);
Consolidated Statement of Earnings javascript:loadReport(2);
Consolidated Statement of Comprehensive Income javascript:loadReport(3);
Consolidated Statement of Financial Position javascript:loadReport(4);
Consolidated Statement of Financial Position (Parenthetical) javascript:loadReport(5);
Consolidated Statement of Cash Flows javascript:loadReport(6);
Consolidated Statement of Changes in Equity javascript:loadReport(7);
Consolidated Statement of Changes in Equity (Parenthetical) javascript:loadReport(8);
Significant Accounting Policies javascript:loadReport(9);
Accounting Changes javascript:loadReport(10);
Acquisitions/Divestitures javascript:loadReport(11);
Financial Instruments javascript:loadReport(12);
Inventories javascript:loadReport(13);
Financing Receivables javascript:loadReport(14);
Propert

In [252]:
page2 = urllib2.urlopen("https://www.sec.gov/Archives/edgar/data/51143/000104746915001106/R2.htm")
soup2 = BeautifulSoup(page2,"lxml")

In [302]:
import re
for f in soup2.find_all(class_="pl"):
    print { f.text : re.search("defref\_((us-gaap|ibm)\_[a-zA-Z]+)",f.a['onclick']).group(1).split("_") }


{u'Revenue': ['us-gaap', 'RevenuesAbstract']}
{u'Services': ['us-gaap', 'SalesRevenueServicesNet']}
{u'Sales': ['us-gaap', 'SalesRevenueGoodsNet']}
{u'Financing': ['us-gaap', 'FinancialServicesRevenue']}
{u'Total revenue (Note T)': ['us-gaap', 'Revenues']}
{u'Cost': ['us-gaap', 'CostOfRevenueAbstract']}
{u'Services': ['us-gaap', 'CostOfServices']}
{u'Sales': ['us-gaap', 'CostOfGoodsSold']}
{u'Financing': ['us-gaap', 'FinancialServicesCosts']}
{u'Total cost': ['us-gaap', 'CostOfRevenue']}
{u'Gross profit': ['us-gaap', 'GrossProfit']}
{u'Expense and other (income)': ['ibm', 'ExpenseAndOtherIncomeAbstract']}
{u'Selling, general and administrative': ['us-gaap', 'SellingGeneralAndAdministrativeExpense']}
{u'Research, development and engineering (Note O)': ['us-gaap', 'ResearchAndDevelopmentExpense']}
{u'Intellectual property and custom development income': ['ibm', 'IntellectualPropertyAndCustomDevelopmentIncome']}
{u'Other (income) and expense': ['ibm', 'OtherIncomeAndExpense']}
{u'Interest

In [148]:
from selenium import webdriver
driver = webdriver.PhantomJS()

In [162]:
driver.get("https://www.sec.gov/cgi-bin/viewer?action=view&cik=51143&accession_number=0001047469-15-001106")

In [166]:
soup = BeautifulSoup(driver.page_source,"lxml")
print soup

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html xmlns="http://www.w3.org/1999/xhtml"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>View Filing Data</title>
<script src="/include/jquery-1.4.3.min.js" type="text/javascript"></script>
<script src="/include/accordionMenu.js" type="text/javascript"></script>
<script src="/include/Show.js" type="text/javascript"></script>
<link href="/include/interactive.css" rel="stylesheet" type="text/css"/>
<link href="/include/report.css" rel="stylesheet" type="text/css"/>
<link href="/include/print.css" media="print" rel="stylesheet" type="text/css"/>
<link href="/include/xbrlViewerStyle.css" rel="stylesheet" type="text/css"/>
<!--[if lt IE 8]>
<style type="text/css">
li.accordion a {display:inline-block;}
li.accordion a {display:block;}
</style>
<![endif]-->
<script language="javascript" type="text/javascript">
var InstanceReportXslt = "/include/InstanceRe

In [154]:
element    = driver.find_element_by_id("menu")
accordians = element.find_elements_by_class_name("accordion")
#accordians.click()
print element.text.split("\n")
len(accordians)
fstatements = filter(lambda x : x.text == "Financial Statements",accordians)[0]
driver.get_screenshot_as_file("/Users/vgenty/Desktop/aho.png")

[u'Cover', u'Financial Statements', u'Notes to Financial Statements', u'Accounting Policies', u'Notes Tables', u'Notes Details', u'All Reports']


True

In [161]:
print fstatements.click()
driver.get_screenshot_as_file("/Users/vgenty/Desktop/aho.png")

None


True

In [169]:
element    = driver.find_element_by_id("menu_cat2")
accordians = element.find_elements_by_class_name("xbrlviewer")
print element.text.split("\n")
len(accordians)

[u'Financial Statements']


0

In [197]:
element    = driver.find_element_by_id("menu")
reports    = element.find_elements_by_class_name("xbrlviewer")

In [220]:
for a in reports: 
    print a.get_attribute("text"), a.get_attribute("href")

Document and Entity Information javascript:loadReport(1);
Consolidated Statement of Earnings javascript:loadReport(2);
Consolidated Statement of Comprehensive Income javascript:loadReport(3);
Consolidated Statement of Financial Position javascript:loadReport(4);
Consolidated Statement of Financial Position (Parenthetical) javascript:loadReport(5);
Consolidated Statement of Cash Flows javascript:loadReport(6);
Consolidated Statement of Changes in Equity javascript:loadReport(7);
Consolidated Statement of Changes in Equity (Parenthetical) javascript:loadReport(8);
Significant Accounting Policies javascript:loadReport(9);
Accounting Changes javascript:loadReport(10);
Acquisitions/Divestitures javascript:loadReport(11);
Financial Instruments javascript:loadReport(12);
Inventories javascript:loadReport(13);
Financing Receivables javascript:loadReport(14);
Property, Plant and Equipment javascript:loadReport(15);
Investments and Sundry Assets javascript:loadReport(16);
Intangible Assets Inclu

In [227]:
driver.execute_script("loadReport(2);")

In [228]:
driver.save_screenshot("/Users/vgenty/Desktop/current.png")

True

In [229]:
soup = BeautifulSoup(driver.page_source,"lxml")

In [234]:
report = soup.find(id="reportDiv")

u'Consolidated Statement of Earnings (USD $)In Millions, except Share data, unless otherwise specified'

In [72]:
accordians

[]

In [ ]:
contexts = {}
for context in xbrl_contexts: 
    period = context.findChildren(name='period')[0]
    
    if period.startDate is not None:
        contexts[context.attrs['id']] = {'period' : {'startDate' : period.startDate.contents[0],
                                                     'endDate' : period.endDate.contents[0]
                                                },
                                          'type'    : 'range'}
    if period.instant is not None:
        contexts[context.attrs['id']] = {'period' : {'Date' : period.instant.contents[0] },
                                          'type'   : 'instant'}

In [ ]:
import pandas as pd
contexts_df=pd.DataFrame(contexts)

In [ ]:
contexts_df

In [ ]:
#aho = soup.findAll(id=re.compile('ID_[0-9]+'))[0]
#aho1 = soup.findAll(lambda tag : tag.prefix == 'dei')
aho2 = soup.findAll(lambda tag : tag.prefix == 'us-gaap')

In [ ]:
document_info = {}
for dei in soup.findAll(lambda tag : tag.prefix == 'dei' and re.search('ID_[0-9]+',tag.attrs['id']) is not None):
    document_info[dei.name] = {'attrs' : dei.attrs , 'value' : dei.contents}

In [ ]:
document_df = pd.DataFrame(document_info)

In [ ]:
document_df

In [ ]:
gaap_info = {}
for gaap in soup.findAll(lambda tag : tag.prefix == 'us-gaap' and re.search('ID_[0-9]+',tag.attrs['id']) is not None and re.search('TextBlock$',tag.name) is not None):
    gaap_info[gaap.name] = {'attrs' : gaap.attrs , 'value' : gaap.contents}

In [ ]:
gaap_df = pd.DataFrame(gaap_info)

In [ ]:
gaap_df

In [ ]:
special_info = {}
for special in soup.findAll(lambda tag : tag.prefix == 'ibm' and re.search('ID_[0-9]+',tag.attrs['id']) is not None):
    special_info[special.name] = {'attrs' : special.attrs , 'value' : special.contents}

In [ ]:
special_df = pd.DataFrame(special_info)

In [ ]:
special_df